# Day 8: Seven Segment Search

In [1]:
from pathlib import Path
import re
from collections import Counter
from itertools import chain
from typing import Callable

from aoc2021.util import read_as_list, invert_dict

## Puzzle input data

In [2]:
parse_input = lambda x: re.split('[\s|]+', x.strip())

# Test data.
tdata = list(map(parse_input, [
    'be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe',
    'edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc',
    'fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg',
    'fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb',
    'aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea',
    'fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb',
    'dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe',
    'bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef',
    'egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb',
    'gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce',
]))

# Input data.
data = read_as_list(Path('./day08-input.txt'), func=parse_input)
data[:2]

[['badc',
  'bd',
  'dbeaf',
  'cfdbge',
  'dfb',
  'cfbdea',
  'efbag',
  'edcfgab',
  'dcafe',
  'degfca',
  'eacfd',
  'acdfbe',
  'cbdegf',
  'fcbaedg'],
 ['cd',
  'fdbac',
  'egcfab',
  'gbadcfe',
  'cfgdeb',
  'cbadfe',
  'deca',
  'cdf',
  'dfabg',
  'abefc',
  'dcf',
  'cfbad',
  'gbafced',
  'fcd']]

## Puzzle answers
### Part 1

In [3]:
def output_digits(line: list[str]) -> list[str]:
    return line[10:]


def is_unique(digit: str) -> bool:
    return len(digit) in {2, 3, 4, 7}


def num_unique_output_digits(data: list[list[str]]) -> int:
    output = map(output_digits, data)
    return sum(len(list(filter(is_unique, ds))) for ds in output)    


assert output_digits(tdata[0]) == ['fdgacbe','cefdb','cefbgd','gcbe']
assert all(map(is_unique, ['ab','abc','abcd','abcdefg']))
assert not any(map(is_unique, ['abcde','abcdef']))
assert num_unique_output_digits(tdata) == 26

In [4]:
n = num_unique_output_digits(data)
print(f'Number of times the digits 1, 4, 7, or 8 appear in the output values: {n}')

Number of times the digits 1, 4, 7, or 8 appear in the output values: 387


### Part 2

In [5]:
SEGMENTS = {
    # easy
    1: 'cf',
    7: 'acf',
    4: 'bcdf',
    8: 'abcdefg',
    # hard
    0: 'abcefg',
    2: 'acdeg',
    3: 'acdfg',
    5: 'abdfg',
    6: 'abdefg',
    9: 'abcdfg',
}
EASY_NUMS = {1, 4, 7, 8}


def possible_matches(digit: str) -> list[int]:
    match len(digit):
        case 2: return [1]
        case 3: return [7]
        case 4: return [4]
        case 6: return [0, 6, 9]
        case 5: return [2, 3, 5]
        case 7: return [8]
        case _: raise Exception(f'invalid digit {digit}')


def easy_digit(n: int, digits: list[str]) -> str:
    if n not in EASY_NUMS:
        raise Exception('number {n} is not easy')
    for d in digits:
        if possible_matches(d)[0] == n:
            return d


def musthave_segments(length: int, digits: list[str]) -> set[str]:
    return set.intersection(*map(set, filter(lambda x: len(x) == length, digits)))


def encoding_from(digits: list[str]) -> dict[str,str]:
    nums = {n: easy_digit(n, digits) for n in EASY_NUMS}
    encoding = {}
    encoding['a'], = set(nums[7]) - set(nums[1])
    encoding['b'], = [k for k, v in Counter(''.join(digits)).items() if v == 6]
    encoding['c'], = set([k for k, v in Counter(''.join(digits)).items() if v == 8]).intersection(nums[1])
    encoding['d'], = set([k for k, v in Counter(''.join(digits)).items() if v == 7]).intersection(nums[4])
    encoding['e'], = [k for k, v in Counter(''.join(digits)).items() if v == 4]
    encoding['f'], = [k for k, v in Counter(''.join(digits)).items() if v == 9]
    encoding['g'], = musthave_segments(5, digits) - set(nums[7]) - set(nums[4])
    return encoding


def decoder(digits: list[str]) -> Callable[[str], int]:
    char_decoding = invert_dict(encoding_from(digits))
    str_decoding = {frozenset(k): v for k, v in invert_dict(SEGMENTS).items()}
    def digit2int(d: str) -> int:
        return str_decoding[frozenset(d.translate(str.maketrans(char_decoding)))]
    return digit2int


def signal_digits(line: list[str]) -> list[str]:
    return line[:10]


def output_values(data: list[list[str]]) -> list[int]:
    return [int(''.join(map(str, map(decoder(signal_digits(line)), output_digits(line))))) for line in data]


assert musthave_segments(2, SEGMENTS.values()) == {'c','f'}
assert musthave_segments(5, SEGMENTS.values()) == {'a','d','g'}
assert signal_digits(tdata[0]) == ['be','cfbegad','cbdgef','fgaecd','cgeb','fdcge','agebfd','fecdb','fabcd','edb']
assert decoder(signal_digits(tdata[0]))('be') == 1
assert list(map(decoder(signal_digits(list(SEGMENTS.values()))), list(SEGMENTS.values()))) == list(SEGMENTS)
assert output_values(tdata) == [8394,9781,1197,9361,4873,8418,4548,1625,8717,4315]
assert sum(output_values(tdata)) == 61229

In [6]:
n = sum(output_values(data))
print(f'Adding up all of the output values: {n}')

Adding up all of the output values: 986034
